In [1]:
import sys
from itertools import combinations

In [2]:
def load_transactions(filename="browsing.txt"):
    """Read transactions from the file; each line is a session (items separated by spaces)."""
    transactions = []
    with open(filename, 'r') as f:
        for line in f:
            items = line.strip().split()
            if items:
                # Remove duplicates per session by converting to a set.
                transactions.append(set(items))
    print(f"DEBUG: Loaded {len(transactions)} transactions from {filename}")
    return transactions
